In [ ]:
# KNN
vizinhos = 3

In [ ]:
# Importando as Bibliotecas Básicas
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# Importando a Biblioteca do KNN
from sklearn.neighbors import KNeighborsClassifier
# Importando a Biblioteca de Acurácia, Matriz de Confusão (em quantos jogos entrou em quantos foi green ou red)
from sklearn.metrics import accuracy_score, confusion_matrix
# Importando a Biblioteca SkLearn para padronizar os dados (entre 0 e 1)
from sklearn.preprocessing import StandardScaler
# Importando a Biblioteca SkLearn para dividir os dados entre treinamento e teste
from sklearn.model_selection import train_test_split

In [ ]:
# spi_matches.csv contains match-by-match SPI ratings and forecasts back to 2016.
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv")
df = df.dropna()

In [ ]:
df = df[['season','date','league','team1','team2','prob1','probtie','prob2','proj_score1','proj_score2','xg1','xg2','score1','score2']]
df = df[(df.league == "Brasileiro Série A")]
df.columns = ['Temporada','Data','Liga','Home','Away','Prob_H','Prob_D','Prob_A','ProjGols_H','ProjGols_A','XG_H','XG_A','FT_Goals_H','FT_Goals_A']
df

In [ ]:
# Criando a Coluna de 0 e 1 para análise do Modelo de Machine Learning
df.loc[(df['FT_Goals_H'] > df['FT_Goals_A']), 'BackHome'] = 1
df.loc[(df['FT_Goals_H'] <= df['FT_Goals_A']), 'BackHome'] = 0

In [ ]:
df

In [ ]:
df1 = df[['Temporada','Data','Liga','Home','Away','Prob_H','Prob_D','Prob_A','ProjGols_H','ProjGols_A','XG_H','XG_A','FT_Goals_H','FT_Goals_A','BackHome']]
df2 = df[['Prob_H','Prob_D','Prob_A','ProjGols_H','ProjGols_A','BackHome']]

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1 = df1.dropna()
# convertendo os arrays (matriz)
# definindo qual coluna é a entrada e qual é a saída
df1_x = df1.iloc[:,0:15].values

# separando os dados para treino e para teste
x_dados_treinamento, x_dados_teste = train_test_split(df1_x, test_size=0.20, random_state = 0)

df1 = pd.DataFrame(x_dados_teste)
df1.columns = ['Temporada','Data','Liga','Home','Away','Prob_H','Prob_D','Prob_A','ProjGols_H','ProjGols_A','XG_H','XG_A','FT_Goals_H','FT_Goals_A','BackHome']

In [ ]:
df2 = df2.dropna()
# convertendo os arrays (matriz)
# definindo qual coluna é a entrada e qual é a saída
df2_x = df2.iloc[:,0:5].values
df2_y = df2.iloc[:,5].values

In [ ]:
df2_x

In [ ]:
df2_y

In [ ]:
# padronizando os dados de entrada
scaler_goals = StandardScaler()
df2_x = scaler_goals.fit_transform(df2_x)

# separando os dados para treino e para teste
x_dados_treinamento, x_dados_teste, y_dados_treinamento, y_dados_teste = train_test_split(df2_x, df2_y, test_size=0.20, random_state = 0)

# Definições de Parâmetros e dados de entrada e resultado para treinamento
knn_dados = KNeighborsClassifier(n_neighbors=vizinhos, metric="minkowski", p=2)
knn_dados.fit(x_dados_treinamento, y_dados_treinamento)

In [ ]:
# Previsões
previsoes_knn = knn_dados.predict(x_dados_teste)

In [ ]:
# taxa de acerto
accuracy_score(y_dados_teste, previsoes_knn)

In [ ]:
# matriz de confusão
confusion_matrix(y_dados_teste, previsoes_knn)
# a11 = coisas que ele preveu como sendo 0 e foi 0 (312 jogos ele preveu que não seria H e não foi H) 
# a21 = coisas que ele preveu como sendo 0 e foi 1 (165 jogos ele preveu que não seria H e foi H)
# a12 = coisas que ele preveu como sendo 1 e foi 0 (125 jogos ele preveu que seria H e não foi H)
# a22 = coisas que ele preveu como sendo 1 e foi 1 (182 jogos ele preveu que seria H e foi H)

In [ ]:
# jogos totais usados para teste
jogos_total_teste = confusion_matrix(y_dados_teste, previsoes_knn)[0][0]+confusion_matrix(y_dados_teste, previsoes_knn)[0][1]+confusion_matrix(y_dados_teste, previsoes_knn)[1][0]++confusion_matrix(y_dados_teste, previsoes_knn)[1][1]
jogos_total_teste

In [ ]:
# quantidade total de entrada
jogos_entrada = confusion_matrix(y_dados_teste, previsoes_knn)[0][1]+confusion_matrix(y_dados_teste, previsoes_knn)[1][1]
jogos_entrada

In [ ]:
# calculando a taxa de entrada
taxa_de_entrada = (jogos_entrada/jogos_total_teste)*100
taxa_de_entrada

In [ ]:
# calculando a taxa de acerto
taxa_de_acerto = (confusion_matrix(y_dados_teste, previsoes_knn)[1][1]/jogos_entrada)*100
taxa_de_acerto

In [ ]:
df1["Previsões"] = previsoes_knn

In [ ]:
df1.loc[(df1['Previsões'] == 1) & (df1['BackHome'] == 1), 'Profit'] = 1
df1.loc[(df1['Previsões'] == 1) & (df1['BackHome'] == 0), 'Profit'] = 0
df1.loc[(df1['Previsões'] == 0) & (df1['BackHome'] == 1), 'Profit'] = 0
df1.loc[(df1['Previsões'] == 0) & (df1['BackHome'] == 0), 'Profit'] = 0

filtro = df1.Previsões == 1
df0 = df1[filtro]

# Ajustando o Índice
df0.reset_index(inplace=True, drop=True)
df0.index = df0.index.set_names(['Nº'])
df0 = df0.rename(index=lambda x: x + 1)

df0['Profit_acu'] = df0.Profit.cumsum()
df0.Profit_acu.plot()
df0.Profit_acu.tail(1)

In [ ]:
df0